In [1]:
%pylab inline
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import itertools,time
import sys, os
from collections import OrderedDict
from copy import deepcopy
import numpy as np
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import itertools
import pickle

Populating the interactive namespace from numpy and matplotlib


/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
dataset_tr = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/train.txt.npy'
data_tr = np.load(dataset_tr)
dataset_te = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/test.txt.npy'
data_te = np.load(dataset_te)
vocab = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/vocab.pkl'
vocab = pickle.load(open(vocab,'r'))

In [3]:
vocab_size=len(vocab)
def onehot(data, min_length):
    return np.bincount(data, minlength=min_length)
data_tr = np.array([onehot(doc.astype('int'),vocab_size) for doc in data_tr if np.sum(doc)!=0])
data_te = np.array([onehot(doc.astype('int'),vocab_size) for doc in data_te if np.sum(doc)!=0])
print data_tr.shape
print data_te.shape


(11258, 1995)
(7487, 1995)


In [4]:
n_samples_tr = 11258
n_samples_te = 7488
docs_tr = data_tr
docs_te = data_te

In [47]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)
def log_dir_init(fan_in, fan_out,topics=50): 
    return tf.log((1.0/topics)*tf.ones([fan_in, fan_out]))

tf.reset_default_graph()
class VariationalAutoencoder(object):
    """ 
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        print 'Learning Rate:', self.learning_rate
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        self.keep_prob = tf.placeholder(tf.float32)
        self.is_training = tf.placeholder(tf.bool, [], name='is_training')
        self.samples = tf.placeholder(tf.int32)
        
        self.h_dim = float(network_architecture["n_z"])
        self.a0 = np.ones((1 , 2)).astype(np.float32)/2
        self.a1 = np.ones((1 , self.h_dim)).astype(np.float32)/self.h_dim
        self.a2 = np.ones((1 , self.h_dim)).astype(np.float32)/self.h_dim
        
        self.mu2a0 = tf.constant((np.log(self.a0).T-np.mean(np.log(self.a0),1)).T)
        self.var2a0 = tf.constant(  ( ( (1.0/self.a0)*( 1 - (2.0/2.0) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a0,1) ).T  )
        
        self.mu2a1 = tf.constant((np.log(self.a1).T-np.mean(np.log(self.a1),1)).T)
        self.var2a1 = tf.constant(  ( ( (1.0/self.a1)*( 1 - (2.0/self.h_dim) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a1,1) ).T  )
        
        self.mu2a2 = tf.constant((np.log(self.a2).T-np.mean(np.log(self.a2),1)).T)
        self.var2a2 = tf.constant(  ( ( (1.0/self.a2)*( 1 - (2.0/self.h_dim) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a2,1) ).T  )
        
        
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.initialize_all_variables()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        # Initialize autoencode network weights and biases
        self.network_weights = self._initialize_weights(**self.network_architecture)
        '''Adding 2 subtopics and 1 super topic.'''
        self.z_mean0,self.z_log_sigma_sq0 = \
            self._recognition_network(self.network_weights["weights_recog"], 
                                      self.network_weights["biases_recog"])
            
        self.z_mean1,self.z_log_sigma_sq1 = \
            self._recognition_network(self.network_weights["sub_01"], 
                                      self.network_weights["biases_sub_01"])
        
        self.z_mean2,self.z_log_sigma_sq2 = \
            self._recognition_network(self.network_weights["sub_01"], 
                                      self.network_weights["biases_sub_01"])

        # Draw one sample z from Gaussian distribution
        '''Adding another noise variable for super toipc: hard coding atm'''
        eps0 = tf.random_normal((self.samples, 2), 0, 1, 
                               dtype=tf.float32)
        n_z = self.network_architecture["n_z"]
        eps1 = tf.random_normal((self.samples, n_z), 0, 1, 
                               dtype=tf.float32)
        eps2 = tf.random_normal((self.samples, n_z), 0, 1, 
                               dtype=tf.float32)
        '''Adding RT for subtopics'''
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(self.z_mean0, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq0)), eps0))))
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z0,
                                                             is_training=self.is_training,updates_collections=None))
        
        self.sigma0 = tf.exp(self.z_log_sigma_sq0)
        
        self.z1 = tf.add(self.z_mean1, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq1)), eps1))
        self.z1 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z1,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma1 = tf.exp(self.z_log_sigma_sq1)
        
        self.z2 = tf.add(self.z_mean2, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq2)), eps2))
        self.z2 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z2,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma2 = tf.exp(self.z_log_sigma_sq2)
        
        '''Adding subtopic reconstruction'''
            
#         self.theta = tf.matmul(self.z0,tf.concat(0,[self.z1, self.z2]))
        
        self.theta = tf.squeeze(tf.map_fn(lambda i: 
                                          tf.matmul(tf.expand_dims(tf.gather(self.z0,i),0),
                                                    tf.concat(0,[tf.expand_dims(tf.gather(self.z1,i),0),
                                                                tf.expand_dims(tf.gather(self.z2,i),0)])),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
            
        self.x_reconstr_mean = \
            self._generator_network(tf.nn.dropout(self.theta, self.keep_prob),
                                    self.network_weights["weights_gener"],
                                    self.network_weights["biases_gener"])
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  
                            n_input, n_z):
        all_weights = dict()
        with tf.variable_scope("super"):
            all_weights['weights_recog'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, 2]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, 2])
            }
            all_weights['biases_recog'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([2], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([2], dtype=tf.float32))
            }
        with tf.variable_scope("sub01"):
            all_weights['sub_01'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, n_z]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, n_z])}
            all_weights['biases_sub_01'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([n_z], dtype=tf.float32))
            }
        with tf.variable_scope("sub02"):
            all_weights['sub_02'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, n_z]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, n_z])
            }
            all_weights['biases_sub_02'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([n_z], dtype=tf.float32))
            }
        with tf.variable_scope("gen"):
            all_weights['weights_gener'] = {
                'h2': tf.Variable(xavier_init(n_z, n_hidden_gener_1))
            }
            all_weights['biases_gener'] = {
                'b2': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
            }
        return all_weights
            
    def _recognition_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        
        z_mean = tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_mean']),
                        biases['out_mean']),is_training=self.is_training,updates_collections=None)
        z_log_sigma_sq = \
            tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
                   biases['out_log_sigma']),is_training=self.is_training,updates_collections=None)     
        
        return (z_mean, z_log_sigma_sq)

    def _generator_network(self,z, weights, biases):
        # Generate probabilistic decoder (decoder network)
#         self.layer_do_0 = tf.nn.dropout(tf.nn.softmax(tf.contrib.layers.batch_norm(z)), self.keep_prob)
        self.layer_do_0 =z
        x_reconstr_mean = tf.add(tf.matmul(self.layer_do_0, 
                                           tf.nn.softmax(tf.contrib.layers.batch_norm(weights['h2'],\
is_training=self.is_training,updates_collections=None))),0.0)
#         x_reconstr_mean = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(
#                     tf.matmul(self.layer_do_0, weights['h2']),0.0)))
    
        return x_reconstr_mean

            
    def _create_loss_optimizer(self):
        
        self.x_reconstr_mean+=1e-10
     
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(self.x_reconstr_mean),1)#/tf.reduce_sum(self.x,1) 

    
        latent_loss0 = 0.5*( tf.reduce_sum(tf.div(self.sigma0,self.var2a0),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a0 - self.z_mean0),self.var2a0),
                  (self.mu2a0 - self.z_mean0)),1) - 2 +\
                           tf.reduce_sum(tf.log(self.var2a0),1)  - tf.reduce_sum(self.z_log_sigma_sq0  ,1) ) 
        
        latent_loss1 = 0.5*( tf.reduce_sum(tf.div(self.sigma1,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z_mean1),self.var2a1),
                  (self.mu2a1 - self.z_mean1)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z_log_sigma_sq1  ,1) )
        
        latent_loss2 = 0.5*( tf.reduce_sum(tf.div(self.sigma2,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean2),self.var2a2),
                  (self.mu2a2 - self.z_mean2)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq2  ,1) )

        
        
#         latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
#                                            - tf.square(self.z_mean) 
#                                            - tf.exp(self.z_log_sigma_sq), 1)


        self.cost = tf.reduce_mean(reconstr_loss) \
    + tf.reduce_mean(latent_loss0 + latent_loss1 + latent_loss2) # average over batch
        
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate,beta1=0.9).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
#         opt, cost,emb,kld = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2'],
#                                           self.kld),feed_dict={self.x: X,self.keep_prob: .9})
        opt,cost,emb = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2']
                                      ),feed_dict={self.x: X,self.keep_prob: .7,self.is_training:True,self.samples:1})
        return cost,emb
    
    def test(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2 = self.sess.run((self.cost,self.z1,self.z2),
                                        feed_dict={self.x: np.expand_dims(X, axis=0),self.keep_prob: 1.0,
                                                   self.is_training:False,self.samples:1})
        return cost,e1,e2
    
    def batchtest(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2 = self.sess.run((self.cost,self.z1,self.z2),
                                        feed_dict={self.x: X,self.keep_prob: 1.0,
                                                   self.is_training:False,self.samples:1})
        return cost,e1,e2


In [89]:
tf.reset_default_graph()
cost_plot=[]
test_batch = create_minibatch(docs_tr.astype('float32'))
def train(network_architecture, learning_rate=0.0001,
          batch_size=100, training_epochs=10, display_step=5):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    emb=0
    avg_kld=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        avg_kld=0.
        total_batch = int(n_samples_tr / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,emb = vae.partial_fit(batch_xs)
            c,a,b = vae.batchtest(batch_xs)

#             sys.exit()
            # Compute average loss
            avg_cost += cost / n_samples_tr * batch_size
            avg_kld += c / n_samples_tr * batch_size
            
            if np.isnan(avg_cost):
                print epoch,i,np.sum(batch_xs,1).astype(np.int),batch_xs.shape
                return vae,emb
#                 sys.exit()
        
        # Display logs per epoch step
        if epoch % display_step == 0:
            cost_plot.append(avg_cost)
            print "Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost)
            print 'avg_cost = ',avg_kld
            test_c=0.
            for i in range(int(n_samples_te / batch_size)):
                c,a,b = vae.batchtest(test_batch.next())
                test_c+=c / n_samples_te * batch_size
            print 'avg_test_cost = ',test_c
    return vae,emb

In [90]:
tf.reset_default_graph()
network_architecture = \
    dict(n_hidden_recog_1=100, # 1st layer encoder neurons
         n_hidden_recog_2=100, # 2nd layer encoder neurons
         n_hidden_gener_1=1995, # 1st layer decoder neurons
         n_input=1995, # MNIST data input (img shape: 28*28)
         n_z=50)  # dimensionality of latent space

batch_size=200
learning_rate=0.001
def create_minibatch(data):
    rng = np.random.RandomState(10)
    
    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
        yield data[ixs]
        
minibatches = create_minibatch(docs_tr.astype('float32'))
vae,emb = train(network_architecture, training_epochs=200,batch_size=batch_size,learning_rate=learning_rate)
# plt.plot(cost_plot)


'''
Change log: decoder has no dropout or additional tranformations
'''

Learning Rate: 0.001
Epoch: 0001 cost= 3318.603450567
avg_cost =  12591126.1308
avg_test_cost =  2019.43181356
Epoch: 0006 cost= 2053.020929041
avg_cost =  1716.22363463
avg_test_cost =  1701.8334674
Epoch: 0011 cost= 1916.104123266
avg_cost =  2274.87858677
avg_test_cost =  1535.79411955
Epoch: 0016 cost= 1859.198265869
avg_cost =  1445.96102537
avg_test_cost =  1389.72198617
Epoch: 0021 cost= 1790.077184263
avg_cost =  1434.62223738
avg_test_cost =  1368.37682479
Epoch: 0026 cost= 1739.282517154
avg_cost =  2061.98986363
avg_test_cost =  1758.78781376
Epoch: 0031 cost= 1588.018051747
avg_cost =  1232.1025578
avg_test_cost =  1191.68164506
Epoch: 0036 cost= 1674.408983646
avg_cost =  2076.73804791
avg_test_cost =  1668.26164539
Epoch: 0041 cost= 1597.443972567
avg_cost =  1445.41125681
avg_test_cost =  1622.31872917
Epoch: 0046 cost= 1542.420632828
avg_cost =  1268.53173941
avg_test_cost =  1320.7271038
Epoch: 0051 cost= 1527.893170499
avg_cost =  1495.68898358
avg_test_cost =  1478.2

'\nChange log: decoder has no dropout or additional tranformations\n'

In [91]:
def print_top_words(beta, feature_names, n_top_words=5):
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))

print_top_words(emb, zip(*sorted(vocab.items(), key=lambda x: x[1]))[0])

motherboard card video upgrade port
god christian jesus bible faith
michael friend picture mike disclaimer
space laboratory datum nasa shuttle
israeli jew israel homosexual jewish
god christian believe keith jon
surrender compound waco gordon gas
science brain treatment msg scientific
thanks hus video anyone driver
space flight orbit spacecraft nasa
chip key phone clipper encryption
armenians kill witness soul turks
pat new university ron june
unix user software ftp information
volume active identify overall disease
exist existence prove atheist assumption
fax email mail address please
signal electronics tube amp voltage
go say know think come
rangers toronto montreal pittsburgh game
government law federal illegal court
windows memory microsoft printer nt
game team playoff score rangers
write think like yeah get
god christian bible believe faith
program file window display format
thanks please anyone hus appreciate
gun clinton criminal texas tax
university institute april research cali

In [93]:

cost=[]
e1=[]
e2=[]
idx=0
for doc in docs_te:
    idx+=1
    doc = doc.astype('float32')
    n_d = np.sum(doc)
    c,a,b=vae.test(doc)
    cost.append(c)
#     print c,n_d
    e1.append(a)
    e2.append(b)
    if idx%1000==0:
        print idx
    
print ((np.mean(np.array(cost))))
# print np.mean(e1,0)
# print np.mean(e2,0)

1000
2000
3000
4000
5000
6000
7000
2013.83


In [95]:
print np.argsort(np.mean(e1,0)[0])
print np.argsort(np.mean(e2,0)[0])
print np.mean(e1,0)#[0]
print np.mean(e2,0)#[0]
print np.exp(np.mean(np.array(cost)))

[14  2 18 42 12 28 46 31 10  6  9 15 17  3 41 22 13 19 48  7 25 32 30 40 21
 34 43 20 37 39 16 47 27 24  0 11  4 45 49 38 35  5 36  1 44 33 23 26 29  8]
[14 42  2 15 10 18 46 12  6 40  9 30 38 45 32 41 39 34 11  7 28  3 36  5 31
 27 19 16 48 17 37 35 13  4 20 21  0 47 25 49 43 33 24 22  1  8 26 29 23 44]
[[ 0.02194456  0.02537401  0.0108702   0.01782281  0.02246     0.02527412
   0.01636191  0.01938766  0.02838152  0.01651694  0.0161326   0.02238633
   0.01337486  0.01914852  0.00881824  0.01656394  0.0206736   0.01703456
   0.01097411  0.01922328  0.02026629  0.01963202  0.01850399  0.0276175
   0.02152281  0.01939556  0.02804637  0.02142353  0.01426757  0.02817168
   0.0194367   0.01583602  0.01942176  0.02656861  0.01987723  0.02465023
   0.02527926  0.02029347  0.02375558  0.02066274  0.01956308  0.01800217
   0.01165098  0.02009346  0.02611873  0.02261652  0.01547787  0.02099394
   0.01933608  0.02278742]]
[[ 0.0223718   0.0263685   0.01101574  0.01919817  0.02067319  0.01930213
 

In [19]:
pickle.dump(cost,open('cost','w'))
pickle.dump(e1,open('e1','w'))
pickle.dump(e2,open('e2','w'))

In [19]:
e=e2
print np.amax(np.mean(e,0))
print np.min(np.mean(e,0))

0.066614
0.00266949


In [98]:
test_batch = create_minibatch(docs_te.astype('float32'))
def test(vae,minibatches):
    emb=0
    avg_kld=0
    print batch_size
    # Training cycle
    for epoch in range(10):
        avg_cost = 0.
        avg_kld = 0.
        total_batch = int(n_samples_te / batch_size)
        print total_batch
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,a,b = vae.batchtest(batch_xs)
            cost=cost
            # Compute average loss
            avg_cost += cost / total_batch#n_samples_tr * batch_size


        # Display logs per epoch step
#         if epoch % display_step == 0:
        cost_plot.append(avg_cost)
        print "Epoch:", '%04d' % (epoch+1), \
              "cost=", "{:.9f}".format(np.exp((avg_cost/90)))
        print 'avg_cost = ',avg_cost

test(vae,test_batch)

200
37
Epoch: 0001 cost= 1951.299849720
avg_cost =  681.862591718
37
Epoch: 0002 cost= 1802.893371419
avg_cost =  674.743327373
37
Epoch: 0003 cost= 2080.210038171
avg_cost =  687.620173274
37
Epoch: 0004 cost= 2131.480287028
avg_cost =  689.811478898
37
Epoch: 0005 cost= 1697.812007509
avg_cost =  669.3386082
37
Epoch: 0006 cost= 1812.600706940
avg_cost =  675.226615287
37
Epoch: 0007 cost= 2147.505452401
avg_cost =  690.485597353
37
Epoch: 0008 cost= 1754.365650309
avg_cost =  672.287635597
37
Epoch: 0009 cost= 1986.632824159
avg_cost =  683.477679278
37
Epoch: 0010 cost= 1301.936128341
avg_cost =  645.444698849


In [65]:
print np.mean(np.sum(docs_tr.astype('float32'),1))

88.0701
